<a href="https://colab.research.google.com/github/AntonSvatko/DSL/blob/main/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import copy

'''
находим полезные символы

Непродуктивным называют нетерминал, из которого нельзя получить цепочку терминалов.
Для поиска в множестве правил непродуктивных нетерминалов используется следующее свойство.

Если все символы правой части правила продуктивны, то продуктивен и символ, стоящий в его левой части.


'''
def productive_symbols(GRAMMAR):
  productive = set()
  size_productive = None

  #Убераем ненужные непродуктивные нетерминалы
  def clear_unproductive_symbols(rule: str):
    return all(map(lambda x: x in productive or x in GRAMMAR['toks'], rule))

  while len(productive) != size_productive:
    size_productive = len(productive)
    productive_s = [var for var, _def in GRAMMAR['vars'].items()
     if list(filter(clear_unproductive_symbols, _def)) != []]
    productive = productive.union(set(productive_s))

  return productive


#Находим все подхотящие нетерминалы
def usefull_symbols(GRAMMAR):
  usefull = {GRAMMAR['hvar']}
  size_productive = None

  while len(usefull) != size_productive:
    size_productive = len(usefull)
    for var, _def in GRAMMAR['vars'].items():
      if var in usefull:
        for rule in _def:
          usefull = usefull.union(set(filter(lambda s: s not in GRAMMAR['toks'], rule)))

  return usefull


#Удаляем бесполезные символы
def clear_useless_symbols(GRAMMAR):
  new_grammer = deepcopy(GRAMMAR)

  productive = productive_symbols(GRAMMAR)
  new_grammer['vars'] = {
      var: [rule for rule in _def 
            if all(map(lambda x: x in productive or x in GRAMMAR['toks'], rule))] 
                          for var, _def in new_grammer['vars'].items()
                          }
  
  usefull = usefull_symbols(new_grammer)
  new_grammer['vars'] = {
      var: [rule for rule in _def 
            if all(map(lambda x: x in usefull or x in GRAMMAR['toks'], rule))] 
                         for var, _def in new_grammer['vars'].items() if var in usefull
                         }
  
  return new_grammer



GRAMMAR = {
      'toks': {(0, ''), (1, 'a'), (2, 'b')},
      'vars': {
          'F': [[(2, 'b'), 'T'],[(0, '') ,'F'], ['L']], 
          'L': [[(0, '')], [(1, 'a')],[(2, '')]], 
          'T': [['T', 'P']],
          'P': []
          },
      'hvar': 'F'
  }
print(clear_useless_symbols(GRAMMAR))


GRAMMAR = clear_useless_symbols(GRAMMAR)

{'toks': {(0, ''), (1, 'a'), (2, 'b')}, 'vars': {'F': [[(0, ''), 'F'], ['L']], 'L': [[(0, '')], [(1, 'a')]]}, 'hvar': 'F'}


Записываем те нетерминалы, в правой части которых стоят только терминалы или исчезающие нетерминалы



In [57]:
def disappear_symbols(GRAMMAR):
  size_productive = None
  needed = set()
  while len(needed) != size_productive:
    size_productive = len(needed)
    needed_symbols = [var for var, definition in GRAMMAR['vars'].items()]
    needed = needed.union(set(needed_symbols))
  return needed


print('Disappear:', str(disappear_symbols(GRAMMAR)))

Disappear: {'L', 'F'}


Out:
{'toks': {(0, ''), (1, 'a'), (2, 'b')}, 'vars': {'F': [[(0, ''), 'F'], ['L']], 'L': [[(0, '')], [(1, 'a')]]}, 'hvar': 'F'}

Disappear: {'L', 'F'}